# Aula 3 - Limpeza e transformação de dados

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Limpeza de Dados (dropna, fillna, isnull, notnull, replace, duplicated, drop_duplicated)
- 2) Transformação de Dados (cut, qcut, get_dummies)

________

### Objetivos

Apresentar como identificamos valores nulos, discutir melhores formas de tratá-los além de trabalhar com variáveis dummies.

________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como identificar valores nulos e como tratá-los.
- Identificar quando utilizar variáveis dummies e como criá-las.
____
____
____

In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
df = pd.read_csv("data/titanic.csv")

## Limpeza de Dados: Dados Faltantes

### `.isna() e isnull()`
retorna um df booleano indicando se existe um nulo naquela posição. `pd.isnull()` é um alias para `pd.isna()` dentro do python como podemos ver na <a href="https://github.com/pandas-dev/pandas/blob/0409521665bd436a10aea7e06336066bf07ff057/pandas/core/dtypes/missing.py#L109">documentação</a>. O pandas dataframe é baseado nos df do R, onde null e na tem sentidos distintos.

In [ ]:
pd.isnull

Tanto o `.isna()` quanto o `.isnull()` retornam um df booleano:

In [ ]:
df.isna()

Para termos a quantidade de nans em cada coluna podemos somar os valores true:

### `.notnull()`

retorna df com True ou False, com True para elementos não nulos

### `.dropna()`

elimina nulos em todo o df ou em colunas especificadas quando utilizado o argumento `subset`. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
df.dropna(subset=['Age','Embarked'])

In [ ]:
df.dropna(subset=['Age','Embarked']).isna().sum()

### `.fillna()`

preeche os valores nulos com valores especificados.
Parâmetros úteis:

   * method {‘bfill’,‘ffill’, None}, default None <br>
       Preenche os nas propagando o último valor válido para frente (ffill) ou utiliza próxima observação válida para preencher os nulos (bfill).
   * axis {0 para ‘index’, 1 para ‘colunas’} <br>
       Eixo no qual se quer preencher os nulos
   * inplace {booleano}, default False


In [ ]:
fill = pd.DataFrame({'original': [np.nan,1,2,np.nan,4,5,np.nan,np.nan, np.nan, 9, 10, np.nan]})
fill

In [ ]:
fill['cte'] = fill.original.fillna(999)
# fill['media'] = fill.original.fillna(fill.original.mean())
# fill['ffill'] = fill.original.fillna(method='ffill')
# fill['ffill_limit'] = fill.original.fillna(method='ffill', limit=2)
# fill['bfill'] = fill.original.fillna(method='bfill')

fill

### Voltando ao dataset do titanic...
Quais soluções poderíamos utilizar para preencher os nulos em Age? <br>
E as cabines nulas? <br>
E os portões de embarque nulos? <br>
Qual os possíveis problemas que podemos encontrar utilizando essas abordagens? <br>


_________
_________
**Exercício:** Ordene as colunas Pclass e Fare, nessa ordem e ascendente, e preencha os nulos utilizando o método foward fill.

_________
_________

### `.interpolate()`
Faz a interpolação dos missing de acordo com o método escolhido.
[Documentação](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.interpolate.html)

In [ ]:
s = pd.DataFrame([0, 1, np.nan, 3, 4, np.nan, 400], columns=['original'])
s['linear'] = s.original.interpolate()
s['segundo_grau'] = s.original.interpolate(method="polynomial", order=2)
s

### `.replace()`

substitui elementos dentro do df. É um dos métodos que aceita o parâmetro `inplace`.


In [ ]:
# para apenas uma troca


In [ ]:
# para mais de uma troca
df.replace({'Sex':{'male':'H', 'female':'M'}})

In [ ]:
df.isna().sum()

___________________
___________________

**Exercício**: Troque os valores abreviados do portão de embarque para seus respectivos nomes 
(C = Cherbourg; Q = Queenstown; S = Southampton) e preencha os dados faltantes com o portão mais frequente

In [ ]:
# Resposta


___________________
___________________


## Limpeza de Dados: Dados duplicados
### `.duplicated()` e `.drop_duplicated()`

O `.duplicated()` retorna uma series indicando se determinada linha possui duplicados ou não. Já o `.drop_duplicated()`, elimina as linhas duplicadas. <br>
Parâmetros do `.drop_duplicated()`:

   * `subset` -  seleciona colunas para serem utilizadas na comparação de linhas duplicadas
   * `keep` - {‘first’, ‘last’, False} (default ‘first’) Determina qual duplicado manter
   * `inplace` - (default False) Se False retorna uma cópia do df com as alterações, se True faz as modificações no próprio df



In [ ]:
# vamos criar um df que possua linhas duplicadas
df_dup = df.copy()

df_dup.loc[891, :] = df_dup.loc[0, :]
df_dup.loc[892, :] = df_dup.loc[1, :]
df_dup.loc[893, :] = df_dup.loc[1, :]

print(df.shape, df_dup.shape)

In [ ]:
df_dup.duplicated().sum()

In [ ]:
print(df_dup.shape)
print(df_dup.drop_duplicates().shape)

_______________________
_______________________
**Exercícios:** Elimine os duplicados considerando as colunas 'brand' e 'style' mantendo o último valor.

__________
__________


## Transformação de dados

### `.get_dummies()`

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
`pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)`

## Material de Aprofundamento

### `.cut()`
O método `.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
`pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)`

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [ ]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

In [ ]:
df.cut_bins.value_counts()

Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [ ]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

In [ ]:
df.cut_bins.unique()

In [ ]:
df.cut_classes.value_counts()

Também podemos passar uma lista com os valores de início e fim dos bins:

In [ ]:
pd.cut(df.Age, [0,20,60,80]).unique()

In [ ]:
df.Age.describe()

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### `.qcut()`
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `.cut()` x `.qcut()`
   * O comando `.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>

[Parâmetros](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html)
<br>
`pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')`

In [ ]:
pd.cut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()

In [ ]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

## Exercícios

1. Converta os países do dataframe abaixo para variáveis dummies eliminando a primeira coluna e utilizando 'Country' como prefixo no nome das colunas.

2. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br> <br>
a. Tem nulos nesse df? Se sim, preencha os nulos da forma que o grupo decidir.

   b. Converta as variáveis categóricas em numéricas.

4. Para esse exercício vamos considerar o dataset https://archive.ics.uci.edu/ml/datasets/automobile que traz a especificação de modelos de carros em 1985.

In [ ]:
# Considere a seguinte lista como o nome das colunas do dataset
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

a. Esse dataset possui nans? Se sim, o grupo precisa entrar em um acordo de como tratá-los.

b. Esse dataset possui linhas duplicadas?

c. O consumo dos automóveis na cidade e rodovia, representados pelas colunas "city-mpg" e "highway-mpg" respectivamente, estão na unidade mpg (miles per gallon). Converta essas colunas para consumo em L/100km sabendo que a conversão é dada por:

L/100km = 235 / mpg 

d. Converta as colunas "body_style" e "drive_wheels" para variáveis dummies.

## Referências:
